Libraries used and initial data retrival

In [5]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt') #gives error if not specified!
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier



def lol2str(doc):
    return " ".join([w for sent in doc for w in sent])

mr = movie_reviews
neg = mr.paras(categories='neg')
pos = mr.paras(categories='pos')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\dalzo\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dalzo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Vectorizers

In [10]:
cv = CountVectorizer()
tv = TfidfVectorizer()

Classifiers

In [11]:
naive_bias = MultinomialNB()
lsvm = SVC(kernel="linear", C=0.025)
svm = SVC()
dl = MLPClassifier(alpha=1, max_iter=1000)

F1 scores before the objectivity removal

In [24]:
print('PRE-REMOVAL','\n','CountVectorizer')
corpus = [lol2str(d) for d in neg] + [lol2str(d) for d in pos]
labels_pre = numpy.array([0] * len(neg) + [1] * len(pos))
vectors_pre = cv.fit_transform(corpus)

scores = cross_validate(naive_bias, vectors_pre, labels_pre, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Naive Bayes Classifier: ",round(average, 3))

scores = cross_validate(lsvm, vectors_pre, labels_pre, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Linear SVM: ",round(average, 3))

scores = cross_validate(svm, vectors_pre, labels_pre, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("SVM: ",round(average, 3))

scores = cross_validate(dl, vectors_pre, labels_pre, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Neural Net: ",round(average, 3))


print('\n', 'TfidfVectorizer')
vectors_pre = tv.fit_transform(corpus)

scores = cross_validate(naive_bias, vectors_pre, labels_pre, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Naive Bayes Classifier: ",round(average, 3))

scores = cross_validate(lsvm, vectors_pre, labels_pre, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Linear SVM: ",round(average, 3))

scores = cross_validate(svm, vectors_pre, labels_pre, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("SVM: ",round(average, 3))

scores = cross_validate(dl, vectors_pre, labels_pre, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Neural Net: ",round(average, 3))

PRE-REMOVAL 
 CountVectorizer
Naive Bayes Classifier:  0.814
Linear SVM:  0.835
SVM:  0.764
Neural Net:  0.839

 TfidfVectorizer
Naive Bayes Classifier:  0.808
Linear SVM:  0.752
SVM:  0.841
Neural Net:  0.846


Function used to remove the objectivity from sentences

In [3]:
def removeObjectivity(corpus, level= 0.4):
    """
    corpus = the dataset that should be a list of reviews where for each review 
    there are list of sentences and fpr the sentences a list of words 
    level = how much objectivity do we want to remove? 
    (0 totaly objective 1 totally subjective) 0.4 as default
    """
    updated_corpus = []
    removed_sentences = 0
    s = 0
    sent= ""
    for review in corpus:
        updated_sentences=[]
        for sentence in review:
            for word in sentence:
                sent += word + " "
            txtblob = TextBlob(sent)
            s += 1
            if (txtblob.sentiment[1] > level):
                updated_sentences.append(sent)
            else:
                removed_sentences += 1
            sent = ""
        if (len(updated_sentences) > 0):
            upd_review = ""
            for sents in updated_sentences:
                upd_review += sents
            updated_corpus.append(upd_review)
    return updated_corpus, removed_sentences, s

Showing what changes after the objectivity is removed. Note that changing the i variable too high could lead to a mismatch between the reviews before and after the removal!

In [22]:
flat_neg = [lol2str(d) for d in neg]
print(flat_neg[875],'\n') 
i = 0.3
print('Sentences to remove with a threshold of ',i,':')
review = TextBlob(flat_neg[875])
for sent in review.sentences:
    if sent.sentiment[1]<i:
        print(str(sent), '->', sent.sentiment[1])
new_neg, n, k = removeObjectivity(neg,i)
print(new_neg[875])


everything in the phantom you have seen many times before and there is nothing new presented here . wincer displays absolutely no skill in setting up an exciting action sequence . billy zane is wooden as the hero . kristy swanson is given very little to do , and does very little with it . treat williams , looking like rhett butler but sounding like mickey mouse , is one of the worst villains i have ever seen in a movie . only catherine zeta jones , as one of williams cohorts turns in a good performance . she has energy and spunk , which the movie needed much more of . oh yeah , the phantom also has a secret identity but this is so poorly played out you won ' t even care . about the only things i can recommend are a good performance by jones , and some colorful scenery . however , if youre looking for a fun family movie , go watch the underrated flipper . this is not a good movie . 

Sentences to remove with a threshold of  0.3 :
billy zane is wooden as the hero . -> 0.0
however , if yo

F1 scores and for threshold=0.4 with all combinations of classifiers and vectorizer

In [36]:
new_corpus_neg, removed_neg, sent_neg = removeObjectivity(neg, 0.4)
new_corpus_pos, removed_pos, sent_pos = removeObjectivity(pos, 0.4)
new_corpus = new_corpus_neg+new_corpus_pos

removed = removed_neg + removed_pos 
sent = sent_neg +  sent_pos
print("Total sentences: ", sent)
print("Removed sentences: ", removed)

labels_post = numpy.array([0] * len(new_corpus_neg) + [1] * len(new_corpus_pos))

print('POST-REMOVAL','\n','CountVectorizer')

vectors_post = cv.fit_transform(new_corpus)

scores = cross_validate(naive_bias, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Naive Bayes Classifier: ",round(average, 3))

scores = cross_validate(lsvm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Linear SVM: ",round(average, 3))

scores = cross_validate(svm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("SVM: ",round(average, 3))

scores = cross_validate(dl, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Neural Net: ",round(average, 3))

print('\n', 'TfidfVectorizer')
vectors_post = tv.fit_transform(new_corpus)

scores = cross_validate(naive_bias, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Naive Bayes Classifier: ",round(average, 3))

scores = cross_validate(lsvm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Linear SVM: ",round(average, 3))

scores = cross_validate(svm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("SVM: ",round(average, 3))

scores = cross_validate(dl, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Neural Net: ",round(average, 3))

Total sentences:  71532
Removed sentences:  35035
POST-RIMOZIONE 
 CountVectorizer
Naive Bayes Classifier:  0.824
Linear SVM:  0.826
SVM:  0.771
Neural Net:  0.832

 TfidfVectorizer
Naive Bayes Classifier:  0.834
Linear SVM:  0.778
SVM:  0.855
Neural Net:  0.853


F1 scores and for threshold=0.3 with all combinations of classifiers and vectorizer

In [47]:
new_corpus_neg, removed_neg, sent_neg = removeObjectivity(neg, 0.3)
new_corpus_pos, removed_pos, sent_pos = removeObjectivity(pos, 0.3)
new_corpus = new_corpus_neg+new_corpus_pos

removed = removed_neg + removed_pos 
sent = sent_neg +  sent_pos
print("Total sentences: ", sent)
print("Removed sentences: ", removed)

labels_post = numpy.array([0] * len(new_corpus_neg) + [1] * len(new_corpus_pos))

print('POST-RIMOZIONE','\n','CountVectorizer')

vectors_post = cv.fit_transform(new_corpus)

scores = cross_validate(naive_bias, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Naive Bayes Classifier: ",round(average, 3))

scores = cross_validate(lsvm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Linear SVM: ",round(average, 3))

scores = cross_validate(svm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("SVM: ",round(average, 3))

scores = cross_validate(dl, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Neural Net: ",round(average, 3))

print('\n', 'TfidfVectorizer')
vectors_post = tv.fit_transform(new_corpus)

scores = cross_validate(naive_bias, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Naive Bayes Classifier: ",round(average, 3))

scores = cross_validate(lsvm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Linear SVM: ",round(average, 3))

scores = cross_validate(svm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("SVM: ",round(average, 3))

scores = cross_validate(dl, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Neural Net: ",round(average, 3))

Total sentences:  71532
Removed sentences:  27413
POST-RIMOZIONE 
 CountVectorizer
Naive Bayes Classifier:  0.823
Linear SVM:  0.82
SVM:  0.768
Neural Net:  0.828

 TfidfVectorizer
Naive Bayes Classifier:  0.833
Linear SVM:  0.782
SVM:  0.844
Neural Net:  0.861


Using Textblob to do the sentiment evaluation   

In [14]:
negative_reviews = [lol2str(d) for d in neg]
positive_reviews = [lol2str(d) for d in pos]
reviews = negative_reviews + positive_reviews
labels = numpy.array([0] * len(negative_reviews) + [1] * len(positive_reviews))

reviews_train, reviews_test, y_train, y_test = train_test_split(
    reviews, labels, test_size=0.25, random_state=8)

test = []
train = []

for i, review in enumerate(reviews_train):
    if(y_train[i] == 0):
        train.append((review, 'neg'))
    else:
        train.append((review, 'pos'))

for i, review in enumerate(reviews_test):
    if(y_test[i] == 0):
        test.append((review, 'neg'))
    else:
        test.append((review, 'pos'))

cl = NaiveBayesClassifier(train)
print("Before removing the objective sentences",cl.accuracy(test))

Before removing the objective sentences 0.802


In [45]:
new_corpus_neg, removed_neg, sent_neg = removeObjectivity(neg, 0.3)
new_corpus_pos, removed_pos, sent_pos = removeObjectivity(pos, 0.3)
new_reviews = new_corpus_neg + new_corpus_pos
labels = numpy.array([0] * len(new_corpus_neg) + [1] * len(new_corpus_pos))

reviews_train, reviews_test, y_train, y_test = train_test_split(
    new_reviews, labels, test_size=0.25, random_state=8)

new_test = []
new_train = []

for i, review in enumerate(reviews_train):
    if(y_train[i] == 0):
        new_train.append((review, 'neg'))
    else:
        new_train.append((review, 'pos'))

for i, review in enumerate(reviews_test):
    if(y_test[i] == 0):
        new_test.append((review, 'neg'))
    else:
        new_test.append((review, 'pos'))

print("After objectivity removal")
new_cl = NaiveBayesClassifier(new_train)
print(new_cl.accuracy(new_test))

After objectivity removal
0.796


These next few sections are for the training of a model for subjectivity detection

Reading the data

In [1]:
file1 = open('Subjectivity Dataset/objective.txt', 'r')
Lines = file1.readlines()
obj = []
for line in Lines:
    obj.append(line)
file2 = open('Subjectivity Dataset/subjective.txt', 'r')
Lines = file2.readlines()
subj = []
for line in Lines:
    subj.append(line)

Creating the model for subjectivity detection

In [6]:
sentences = obj + subj
labels = numpy.array([0] * len(obj) + [1] * len(subj))
sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, labels, test_size=0.25, random_state=8)

vectorizer = TfidfVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)


classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.8972


New method for removng objective sentences

In [7]:
def newRemoveObjectivity(corpus, model, vectorizer):
    """
    corpus = the dataset that should be a list of reviews where for each review 
    model = the already trained model to categorize a sentece being subjective or objective
    vectorizer = this should be the same as the one used to train the previous model
    """
    updated_corpus = []
    removed_sentences = 0
    s = 0
    sent= ""
    for review in corpus:
        updated_sentences=[]
        for sentence in review:
            x = []
            for word in sentence:
                sent += word + " "
            x.append(sent)
            x = vectorizer.transform(x)
            prediction = model.predict(x)
            s += 1
            if (prediction == [1]): #meaning that the sentence is considered subjective
                updated_sentences.append(sent)
            else:
                removed_sentences += 1
            sent = ""
        if (len(updated_sentences) > 0):
            upd_review = ""
            for sents in updated_sentences:
                upd_review += sents
            updated_corpus.append(upd_review)
    return updated_corpus, removed_sentences, s

Creating the new dataset and computing number of removed sentences to compare with previous method

In [8]:
new_corpus_neg, removed_neg, sent_neg = newRemoveObjectivity(neg, classifier, vectorizer)
new_corpus_pos, removed_pos, sent_pos = newRemoveObjectivity(pos, classifier, vectorizer)
removed = removed_neg + removed_pos 
sent = sent_neg +  sent_pos
print("Total sentences: ", sent)
print("Removed sentences: ", removed)

Total sentences:  71532
Removed sentences:  25156


Computing the F1 scrores with different classifiers without using TextBlob 

In [12]:
labels_post = numpy.array([0] * len(new_corpus_neg) + [1] * len(new_corpus_pos))
new_corpus = new_corpus_neg+new_corpus_pos
print('Without using TextBlob\nTfidfVectorizer')
vectors_post = tv.fit_transform(new_corpus)

scores = cross_validate(naive_bias, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Naive Bayes Classifier: ",round(average, 3))

scores = cross_validate(lsvm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Linear SVM: ",round(average, 3))

scores = cross_validate(svm, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("SVM: ",round(average, 3))

scores = cross_validate(dl, vectors_post, labels_post, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
average = sum(scores['test_f1_micro'])/len(scores['test_f1_micro'])
print("Neural Net: ",round(average, 3))

Without using TextBlob
TfidfVectorizer
Naive Bayes Classifier:  0.833
Linear SVM:  0.781
SVM:  0.864
Neural Net:  0.868
